In [60]:
from elasticsearch import Elasticsearch
import warnings
warnings.filterwarnings("ignore")

# connect to your elasticsearch address and port
es = Elasticsearch("http://localhost:9200")

# check the index list
index_list = es.cat.indices(format="json")
print(index_list)

[{'health': 'green', 'status': 'open', 'index': '.geoip_databases', 'uuid': 'K_Zhjx6vQASeqBzyRFxNNw', 'pri': '1', 'rep': '0', 'docs.count': '33', 'docs.deleted': '0', 'store.size': '31.1mb', 'pri.store.size': '31.1mb'}, {'health': 'green', 'status': 'open', 'index': '.apm-custom-link', 'uuid': '3ldJVeZfQ1a6Ydo9dB6v2w', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '226b', 'pri.store.size': '226b'}, {'health': 'green', 'status': 'open', 'index': '.apm-agent-configuration', 'uuid': 'RDTlbuKWRJeUY007cc9NNA', 'pri': '1', 'rep': '0', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '226b', 'pri.store.size': '226b'}, {'health': 'green', 'status': 'open', 'index': '.kibana_task_manager_7.17.4_001', 'uuid': '40_U0RooTrGzevEiI9SEUw', 'pri': '1', 'rep': '0', 'docs.count': '17', 'docs.deleted': '4167', 'store.size': '592.6kb', 'pri.store.size': '592.6kb'}, {'health': 'green', 'status': 'open', 'index': '.kibana_7.17.4_001', 'uuid': 'Cmw8DhEJQOSLyn9peAQ34w', 'p

In [87]:
# modify your index_settings
index_settings = {
    "settings": {
        "number_of_shards": 3,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "id": {
                "type": "keyword"
            },
            "name": {
                "type": "text"
            },
            "course": {
                "type": "text"
            },
            "mark": {
                "type": "short"
            }
        }
    }
}

# create index
index_name = "students"
es.indices.create(index=index_name, body=index_settings)
index_list = es.cat.indices(format="json")
info = [index for index in index_list if index["index"] == "students"]
print(info)
# get mapping of index
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

[{'health': 'yellow', 'status': 'open', 'index': 'students', 'uuid': 'vzBtyAopTbGmCV-wQ8Wseg', 'pri': '3', 'rep': '1', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '678b', 'pri.store.size': '678b'}]
{'students': {'mappings': {'properties': {'course': {'type': 'text'}, 'id': {'type': 'keyword'}, 'mark': {'type': 'short'}, 'name': {'type': 'text'}}}}}


In [86]:
# delete index
es.indices.delete(index='students', ignore=[400, 404])
index_list = es.cat.indices(format="json")
info = [index for index in index_list if index["index"] == "students"]
print(info)

[]


In [88]:
# determine the query rules
query = {"query": {"match_all": {}}}

result = es.search(index=index_name, body=query, size=10000) 
print(result)

{'took': 0, 'timed_out': False, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [89]:
document_data = {
    "name": "John Smith",
    "course": "Cloud Computing",
    "mark": 80
}

# insert document using auto id
es.index(index=index_name, body=document_data)

ObjectApiResponse({'_index': 'students', '_type': '_doc', '_id': '0nrMHY8B9-nlTsXci5oo', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [90]:
document_data = {
    "name": "Jane Doe",
    "course": "Cloud Computing",
    "mark": 90
}

# insert document using specified id
es.index(index=index_name, body=document_data, id="my_own_id")

ObjectApiResponse({'_index': 'students', '_type': '_doc', '_id': 'my_own_id', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1})

In [91]:
document_data = {
    "name": "Jane John",
    "course": "Cloud Computing",
    "mark": 100
}

# insert document using auto id
es.index(index=index_name, body=document_data)

ObjectApiResponse({'_index': 'students', '_type': '_doc', '_id': '03rMHY8B9-nlTsXc1JrI', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [112]:
# define your query
query = {
    "query": {
        "match": {
            "course": "cloud"
        }
    }
}

# execute the query
result = es.search(index=index_name, body=query)
for hit in result['hits']['hits']:
    print(hit['_source'])

{'name': 'Jane John', 'course': 'Cloud Computing', 'mark': 100}
{'name': 'John Smith', 'course': 'Cloud Computing', 'mark': 80}
{'name': 'Jane Doe', 'course': 'Cloud Computing', 'mark': 90}


In [93]:
# define your query
query = {
    "query": {
        "match": {
            "name": "Jane"
        }
    }
}

# execute the query
result = es.search(index=index_name, body=query)
for hit in result['hits']['hits']:
    print(hit['_source'])

{'name': 'Jane Doe', 'course': 'Cloud Computing', 'mark': 90}
{'name': 'Jane John', 'course': 'Cloud Computing', 'mark': 100}


In [98]:
# define your query
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "name": "Doe"
                    }
                },
                {
                    "match": {
                        "name": "Smith"
                    }
                }
            ]
        }
    }
}

# execute the query
result = es.search(index=index_name, body=query)
for hit in result['hits']['hits']:
    print(hit['_source'])

In [99]:
# define your query
query = {
    "query": {
        "bool": {
            "should": [
                {
                    "match": {
                        "name": "Doe"
                    }
                },
                {
                    "match": {
                        "name": "Smith"
                    }
                }
            ]
        }
    }
}

# execute the query
result = es.search(index=index_name, body=query)
for hit in result['hits']['hits']:
    print(hit['_source'])

{'name': 'John Smith', 'course': 'Cloud Computing', 'mark': 80}
{'name': 'Jane Doe', 'course': 'Cloud Computing', 'mark': 90}


In [129]:
# define your query
query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match_phrase": {
                        "name": 'Jane John'
                    }
                },
                {
                    "range": {
                        "mark": {
                            "lte": 85
                        }
                    }
                }
            ]
        }
    }
}

# execute the query
result = es.search(index=index_name, body=query)
for hit in result['hits']['hits']:
    print(hit['_source'])